In [1]:
from glob import glob
from bs4 import BeautifulSoup
import os
import numpy as np; import pandas as pd

## PARSING XML FILE
### Read DOC filenames

### parsing all text from each xml 

## Reading xlsx with text examples

In [2]:
#df_keys = pd.DataFrame(columns=('filename', 'text', 'keywords'))
fname = "/Users/cavaste/GoogleDriveCGnal/Projects/Dedalus/dedalus_text_sample_used.xlsx"

df_text_keyw = pd.read_excel(fname, thousands = ',', parse_cols = 'A:C')
df_text_keyw.head()

,Filename,ID,Text
0,table_export_DATA44e8cdc0-0159-1000-8001-c0a80...,DH_AC605550710_1500,"Primary, Malignant neoplasm of prostate"
1,table_export_DATA44e8cdc0-0159-1000-8001-c0a80...,DH_AC605550710_1527,"Secondary, Anemia in chronic kidney disease"
2,table_export_DATA44e8cdc0-0159-1000-8001-c0a80...,DH_AC605550710_1521,"Secondary, Iron deficiency anemia, unspecified"
3,table_export_DATA44e8d2c1-0159-1000-8002-c0a80...,DH_MC876686273_17,"Primary, Stage II, Malignant neoplasm of recto..."
4,table_export_DATA44e8d4a6-0159-1000-8003-c0a80...,DH_Cb671607614_1393,"Primary, Malign Neoplasm Ovarian"


In [3]:
from sklearn.feature_extraction import stop_words
stopwords_ = set(stop_words.ENGLISH_STOP_WORDS)

from nltk.corpus import stopwords

stopwords_.update(stopwords.words('english'))
stopwords_.update(stopwords.words('italian'))

In [4]:
df_text_keyw["Text"] = df_text_keyw["Text"].apply(lambda x: x.lower())

### Counting the number of words in each text

In [5]:
df_text_keyw.insert(3, "Word Count", df_text_keyw["Text"].apply(lambda x: len(x.split())))

### Keyword extraction from each text by using graph

In [6]:
from gensim.summarization import keywords
ratio_value = 0.9

keys_list = []

for i,doc in enumerate(df_text_keyw["Text"]):
#    print(i)
#    print(doc)
    keys = keywords(doc,
                    split=False,
                    ratio=ratio_value,
                    scores=True,
                    lemmatize=False)#,
                    #pos_filter=['NN','JJ','NNS','NNP','NNPS'])
    
    keys = [key for key in keys if key[0] not in stopwords_][:30]    
    keys_list.append(keys) 

In [7]:
df_text_keyw.insert(4,"Keywords Raw Text",keys_list)# = [filenames[i], doc, keys] 

### POS tagging

In [8]:
import nltk
df_text_keyw.insert(5, "POStagged Text", df_text_keyw["Text"].apply(lambda x: nltk.pos_tag(nltk.word_tokenize(x))))

In [9]:
pos_filter=['FW','RB','JJ','NN','NNS','NNP','NNPS']
df_text_keyw.insert(6, "Filtered Text", df_text_keyw["POStagged Text"].
                    apply(lambda x: [word[0] for word in x if word[1] in pos_filter]))

In [10]:
df_text_keyw["Filtered Text"]

0              [primary, malignant, neoplasm, prostate]
1         [secondary, anemia, chronic, kidney, disease]
2     [secondary, iron, deficiency, anemia, unspecif...
3     [primary, stage, ii, malignant, neoplasm, rect...
4                  [primary, malign, neoplasm, ovarian]
5     [secondary, encounter, antineoplastic, chemoth...
6           [20, mg, tablet, crush, resistant, release]
7           [30, mg, tablet, crush, resistant, release]
8                  [primary, unspecified, lump, breast]
9                [secondary, lymph, nodes, unspecified]
10    [helicobacter, pylori, [, h., pylori, ], cause...
11          [primary, coagulation, defect, unspecified]
12    [primary, cerebral, infarction, due, unspecifi...
13    [secondary, iron, deficiency, anemia, unspecif...
14    [primary, stage, iiib, malignant, neoplasm, co...
15         [smil, ct, thorax, abd/pelv, tith, contrast]
16    [primary, malignant, neoplasm, right, main, br...
17    [amitriptyline, hcl, 50mg, oral, tablet, h

## Exporting df to csv

In [11]:
output_filename = "/Users/cavaste/GoogleDriveCGnal/Projects/Dedalus/docs_keywords_ratio"+str(ratio_value)+".csv"
df_text_keyw.to_csv(path_or_buf = output_filename, sep=',',
                    index=False,
                    encoding="utf8")

### Selecting short texts

In [19]:
df_text_short = df_text_keyw[df_text_keyw["Word Count"] < 25]

### Aggregating short texts

In [20]:
df_text_short_grouped = df_text_short.groupby(df_text_short["Filename"]).agg(lambda x: x.tolist())

In [21]:
df_text_short_grouped.head()

,ID,Text,Word Count,Keywords Raw Text,POStagged Text,Filtered Text
Filename,,,,,,
table_export_DATA44e8cdc0-0159-1000-8001-c0a80a0a809c.ldr,"[DH_AC605550710_1500, DH_AC605550710_1527, DH_...","[primary, malignant neoplasm of prostate, seco...","[5, 6, 5]","[[(primary malignant, [ 0.6121701])], [(chroni...","[[(primary, JJ), (,, ,), (malignant, JJ), (neo...","[[primary, malignant, neoplasm, prostate], [se..."
table_export_DATA44e8d2c1-0159-1000-8002-c0a80a0a809c.ldr,[DH_MC876686273_17],"[primary, stage ii, malignant neoplasm of rect...",[8],"[[(primary stage, [ 0.40824829]), (malignant n...","[[(primary, JJ), (,, ,), (stage, NN), (ii, NN)...","[[primary, stage, ii, malignant, neoplasm, rec..."
table_export_DATA44e8d4a6-0159-1000-8003-c0a80a0a809c.ldr,"[DH_Cb671607614_1393, DH_DK39300751_1651, MH_D...","[primary, malign neoplasm ovarian, secondary, ...","[4, 5, 6, 6]","[[(primary malign neoplasm, [ 0.52677807])], [...","[[(primary, JJ), (,, ,), (malign, JJ), (neopla...","[[primary, malign, neoplasm, ovarian], [second..."
table_export_DATA44e8d665-0159-1000-8004-c0a80a0a809c.ldr,[DH_KP69546999_1216],"[primary, unspecified lump in breast]",[5],"[[(primary unspecified, [ 0.6121701])]]","[[(primary, JJ), (,, ,), (unspecified, JJ), (l...","[[primary, unspecified, lump, breast]]"
table_export_DATA44e8d742-0159-1000-8005-c0a80a0a809c.ldr,"[DH_AC112325761_833, Reason For Visit]","[secondary, enlarged lymph nodes, unspecified,...","[5, 11]","[[(nodes, [ 0.5312606]), (secondary enlarged l...","[[(secondary, JJ), (,, ,), (enlarged, VBD), (l...","[[secondary, lymph, nodes, unspecified], [heli..."


In [22]:
df_text_short_grouped["Text"] = df_text_short_grouped["Text"].apply(lambda x: ' '.join(x).replace(",",""))

In [23]:
keys_list = []

for i,doc in enumerate(df_text_short_grouped["Text"]):
#    print(i)
#    print(doc)
    keys = keywords(doc,
                    split=False,
                    ratio=ratio_value,
                    scores=True,
                    lemmatize=False)#,
                    #pos_filter=['NN','JJ','NNS','NNP','NNPS'])
    
    keys = [key for key in keys if key[0] not in stopwords_][:30]    
    keys_list.append(keys) 
    

In [24]:
df_text_short_grouped.insert(3,"Keywords grouped texts",keys_list)# = [filenames[i], doc, keys] 

## Exporting df grouped short to csv

In [25]:
output_filename = "/Users/cavaste/GoogleDriveCGnal/Projects/Dedalus/grouped_docs_keywords_ratio"+str(ratio_value)+".csv"
df_text_short_grouped.to_csv(path_or_buf = output_filename, sep=';',
                             index=False,
                             encoding="utf8")